# FLOW SIMULATION

<img src='imgs/Slide1.png' width='800' align='left' />

#### This notebook simulates the flow of data
1. User submits a search parameters in a web form
2. Search is performed in ElasticSearch index
3. Search results are saved in PostgreSQL
4. Report is generated and saved in S3
5. User consumes report for analysis

# 1. FORM SUBMISSION

<img src='imgs/Slide2.png' width='500' align='left' />

### parameters -- airport & date range

In [252]:
# airport = MSY
airport = 'MSY'

In [253]:
# date range - April 13-20, 2019
date_start = datetime.date(2019, 4, 13)
date_end = datetime.date(2019, 4, 20)
date_range = (str(date_start), str(date_end))
date_range

('2019-04-13', '2019-04-20')

### generate GUID

In [254]:
import uuid 
GUID = str(uuid.uuid1())
GUID

'0c6c4ab0-3593-11ea-8d23-acde48001122'

### JSON wrap

In [255]:
params = {
    'airport' : airport,
    'date_range' : date_range,
    'guid' : GUID
}
params

{'airport': 'MSY',
 'date_range': ('2019-04-13', '2019-04-20'),
 'guid': '0c6c4ab0-3593-11ea-8d23-acde48001122'}

# 2. Lambda <-> ElasticSearch

<img src='imgs/Slide3.png' width='500' align='left' />

### search index

In [256]:
results = !wget http://localhost:9200/itens/_search\?pretty\=true\&q\=airport:{airport} -q -O -

### format results

In [257]:
import json
from pandas.io.json import json_normalize

results_str = ''

for i in results:
    results_str += i 

obj = json.loads(results_str)

### insert GUID

In [258]:
for hit in obj['hits']['hits']:
    hit['_source'].update({'guid' : GUID})    

### convert to dataframe

In [259]:
cols = {
    '_index':'idx',
    '_type':'src_type',
    '_id':'idx_id',
    '_score':'search_score',
    '_source.@timestamp':'idx_timestamp',
    '_source.leave':'leave',
    '_source.name':'name',
    '_source.land':'land',
    '_source.id':'prsn_id',
    '_source.age':'prsn_age',
    '_source.airport':'airport',
    '_source.guid':'guid'
}


df = json_normalize(obj['hits']['hits']).rename(columns=cols)

df['leave'] = df['leave'].astype(str).astype(int)
df['land'] = df['land'].astype(str).astype(int)
df['prsn_id'] = df['prsn_id'].astype(str).astype(int)
df['prsn_age'] = df['prsn_age'].astype(str).astype(int)

df['idx_timestamp'] = pd.to_datetime(df['idx_timestamp'])

# 3. Lambda <-> PostgreSQL

<img src='imgs/Slide4.png' width='500' align='left' />

In [261]:
from sqlalchemy import create_engine
engine = create_engine('postgresql+psycopg2://admin@localhost:5432/trvl')
engine.table_names()

['index_search_results']

In [262]:
table_name = 'index_search_results'
con = engine.connect()
df.to_sql(table_name, con, if_exists='append')
con.close()

In [263]:
engine.table_names()

['index_search_results']

In [264]:
# pd.read_sql_query('select * from ' + table_name, con=engine)

# 4. Lambda -> S3

<img src='imgs/Slide5.png' width='500' align='left' />

In [265]:
results_file_name = 'results_' + GUID + '.csv'

In [266]:
df.to_csv(results_file_name)

In [267]:
!ls   -l {results_file_name}

-rw-r--r--  1 wihill  staff  816 Jan 12 18:28 results_0c6c4ab0-3593-11ea-8d23-acde48001122.csv


In [268]:
!wc   -l {results_file_name}

       6 results_0c6c4ab0-3593-11ea-8d23-acde48001122.csv


In [269]:
!head -4 {results_file_name}

,idx,src_type,idx_id,search_score,idx_timestamp,leave,name,land,prsn_id,prsn_age,airport,guid
0,itens,_doc,COx9lm8BWea_MqX5V5_4,1.3397744,2019-04-02 04:00:00+00:00,1554696000,Blair,1554177600,4,22,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
1,itens,_doc,D-x9lm8BWea_MqX5V5_4,1.3397744,2019-04-14 04:00:00+00:00,1555214400,Blair,1555214400,11,47,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
2,itens,_doc,FOx9lm8BWea_MqX5V5_4,1.3397744,2019-04-14 04:00:00+00:00,1556078400,Beth,1555214400,16,54,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122


# 5. User consume report for analysis

<img src='imgs/Slide6.png' width='500' align='left' />

In [270]:
df

,idx,src_type,idx_id,search_score,idx_timestamp,leave,name,land,prsn_id,prsn_age,airport,guid
0,itens,_doc,COx9lm8BWea_MqX5V5_4,1.339774,2019-04-02 04:00:00+00:00,1554696000,Blair,1554177600,4,22,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
1,itens,_doc,D-x9lm8BWea_MqX5V5_4,1.339774,2019-04-14 04:00:00+00:00,1555214400,Blair,1555214400,11,47,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
2,itens,_doc,FOx9lm8BWea_MqX5V5_4,1.339774,2019-04-14 04:00:00+00:00,1556078400,Beth,1555214400,16,54,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
3,itens,_doc,Fux9lm8BWea_MqX5V5_4,1.339774,2019-04-22 04:00:00+00:00,1555905600,Bill,1555905600,18,61,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
4,itens,_doc,F-x9lm8BWea_MqX5V5_4,1.339774,2019-04-14 04:00:00+00:00,1555300800,Bubba,1555214400,19,61,MSY,0c6c4ab0-3593-11ea-8d23-acde48001122
